In [1]:
'''reading the sudoku problem from file'''

arq = open('sudoku', 'r')
sudoku = arq.readlines()
grid = []

N = 0
for i in range(len(sudoku)):
    
    if(i == 0):
        N = sudoku[i]
    else:
        grid.append(sudoku[i].split())

arq.close()


def show_grid():
    for i in range(len(grid)):          
        print(grid[i])
    print("\n")

In [2]:
'''generate the list with the blank spaces of the grid '''

def empty_list(grid):
    
    empty = {}
    k = 0
    for i in range(len(grid)):
        for j in range(len(grid)):
            if(grid[i][j] == "0"):
                empty[k] = [i,j]
                k = k + 1
            
    return empty
    

In [3]:
'''verification of lin, col, subgrids'''

def check_lin(grid, i, chosen):
    
    if(chosen in grid[i]):
        return False
    else:
        return True
    
def check_col(grid, j, chosen):
    
    for i in range(len(grid)):
        if(chosen == grid[i][j]):
            return False
    
    return True

def check_subgrid(grid, i, j, chosen):
    
    subl = i - i%3
    subc = j - j%3
    aux = subc    
        
    for subl in range(subl, subl+3):
        for subc in range(subc, subc+3):
            if(grid[subl][subc] == chosen):
                return False
         
        subc = aux
        
    return True
    
    

In [4]:
''' backtracking algorithm '''

def backtrack(grid):
    
    empty = empty_list(grid)
    
    if (empty == {}):
        return True
            
    spoti, spotj = empty[0][0], empty[0][1]

    for i in range(1,10):

        chosen = str(i)
        
        #print("Posição: " + str(spoti) + ", " + str(spotj) + " = " + str(i))

        if(check_lin(grid, spoti, chosen) == True and check_col(grid, spotj, chosen) == True and   
           check_subgrid(grid, spoti, spotj, chosen) == True):

            grid[spoti][spotj] = chosen

            result = backtrack(grid)

            if(result == False):
                grid[spoti][spotj] = "0"
            else:
                return True
        

    return False    
    

In [5]:
show_grid()
backtrack(grid)
show_grid()



['3', '0', '6', '5', '0', '8', '4', '0', '0']
['5', '2', '0', '0', '0', '0', '0', '0', '0']
['0', '8', '7', '0', '0', '0', '0', '3', '1']
['0', '0', '3', '0', '1', '0', '0', '8', '0']
['9', '0', '0', '8', '6', '3', '0', '0', '5']
['0', '5', '0', '0', '9', '0', '6', '0', '0']
['1', '3', '0', '0', '0', '0', '2', '5', '0']
['0', '0', '0', '0', '0', '0', '0', '7', '4']
['0', '0', '5', '2', '0', '6', '3', '0', '0']


['3', '1', '6', '5', '7', '8', '4', '9', '2']
['5', '2', '9', '1', '3', '4', '7', '6', '8']
['4', '8', '7', '6', '2', '9', '5', '3', '1']
['2', '6', '3', '4', '1', '5', '9', '8', '7']
['9', '7', '4', '8', '6', '3', '1', '2', '5']
['8', '5', '1', '7', '9', '2', '6', '4', '3']
['1', '3', '8', '9', '4', '7', '2', '5', '6']
['6', '9', '2', '3', '5', '1', '8', '7', '4']
['7', '4', '5', '2', '8', '6', '3', '1', '9']


